# Imports 

In [116]:
!pip install ccxt pandas numpy

In [117]:
!pip install ta-lib  

In [118]:
!pip install ta-lib talib-binary 
!pip install pandas-ta    

ERROR: Could not find a version that satisfies the requirement talib-binary (from versions: none)
ERROR: No matching distribution found for talib-binary


In [119]:
!pip uninstall logging

In [120]:
!pip install asyncio websocket-client
# !pip install sqlite3 
# !pip install logging  

In [121]:
!pip list

Package                   Version
------------------------- -----------
aiodns                    3.5.0
aiohappyeyeballs          2.6.1
aiohttp                   3.12.15
aiosignal                 1.4.0
altair                    5.5.0
asttokens                 3.0.0
async-timeout             5.0.1
asyncio                   4.0.0
attrs                     25.3.0
backcall                  0.2.0
blinker                   1.9.0
build                     1.3.0
cachetools                6.2.0
ccxt                      4.5.2
certifi                   2025.8.3
cffi                      1.17.1
charset-normalizer        3.4.3
click                     8.1.8
colorama                  0.4.6
comm                      0.2.3
contourpy                 1.3.0
cryptography              45.0.6
cycler                    0.12.1
dash                      3.2.0
debugpy                   1.8.16
decorator                 5.2.1
executing                 2.2.0
Flask                     3.1.2
fonttools             

In [122]:
!pip install plotly dash          
!pip install matplotlib seaborn   
!pip install streamlit 

In [123]:
!pip install python-dotenv       
!pip install schedule             
!pip install requests aiohttp     

^C


In [ ]:
!pip install scikit-learn

# Get Data

In [1]:
import os 
from dotenv import load_dotenv

In [2]:
load_dotenv()

False

In [3]:
class Configur():
    EXCHANGE  = "binance"
    SYMBOL = "BTC/USDT"
    TIMEFRAME = "1h"


    API_KEY = ""
    SECRET_KEY = ""
    SANDBOX = ""

    INIT_BALANCE = 10000
    RISK = 0.01
    STOP_LOSS = 0.02
    PROFIT_TAKE = 0.04
    # Strategy Parameters
    RSI_PERIOD = 14
    RSI_OVERSOLD = 30
    RSI_OVERBOUGHT = 70
    EMA_SHORT = 20
    EMA_LONG = 50
    
    # Execution Settings
    MIN_TRADES_FOR_METRICS = 50
    UPDATE_INTERVAL = 60  # seconds
    
    # Logging
    LOG_FILE = 'trades.csv'
    DB_FILE = 'trading_bot.db'



In [4]:
import ccxt
import pandas as pd
import numpy as np
import asyncio
import time
from datetime import datetime
from Config import Configs

In [5]:
import ccxt
import pandas as pd
import numpy as np
import asyncio
import time
from datetime import datetime
from Config import Configs

class Manage_data:
    def __init__(self):
        self.exchange = self._init_exchange()
        self.current_price = 0
        self.ohlcv_data = pd.DataFrame()
        
    def _init_exchange(self):
        exchange_class = getattr(ccxt, 'binance')
        exchange = exchange_class({
            'apiKey': Configs.API_KEY,
            'secret': Configs.SECRET_KEY,
            'sandbox': Configs.SANDBOX,
            'enableRateLimit': True,
        })
        return exchange      
    
    def fetch_ohlcv(self, limit =800):
        try:
            ohlcv = self.exchange.fetch_ohlcv(
                "BTC/USDT",
                "1h",
                limit =limit
            )
            df = pd.DataFrame(ohlcv,columns=['timestamp','open','high','low','close','volume'])
            df.set_index('timestamp', inplace=True)
            self.ohlcv_data = df
            self.current_price = df['close'].iloc[-1]
            return df
        except Exception as e:
            print(e)
            return None
    def get_current_price(self):
        try :
            ticker = self.exchange.fetch_ticker("BTC/USDT")
            self.current_price = ticker['last']
            return self.current_price
        except Exception as e:
            print(f" problem in get_current_price {e}")
            return self.current_price
    def get_balance(self):
        try:
            balance = self.exchange.fetch_balance()
            return  balance['USDT']['free'] if 'USDT' in balance else Configur.INIT_BALANCE
        except Exception as e:
            print( f" problem in get_balance {e}")
            return Configur.INIT_BALANCE

# Make Dataframe

In [6]:
data = Manage_data()
df = data.fetch_ohlcv()
print(df)

                    open       high        low      close    volume
timestamp                                                          
1754017200000  115966.12  116464.26  115621.74  115652.51   5.20842
1754020800000  115648.03  116223.26   88888.00  115712.88   9.60859
1754024400000  115695.53  116255.26   33000.00  115544.20  15.03012
1754028000000  115544.20  200000.00   69374.87  115156.52   9.05393
1754031600000  115156.52  120000.00   77777.00  115090.33   4.78754
...                  ...        ...        ...        ...       ...
1756879200000  110685.57  110984.83  110473.86  110719.99  22.30021
1756882800000  110723.99  111962.80  110520.18  111099.88  26.37469
1756886400000  111099.88  200000.00  110803.87  111019.99  35.81016
1756890000000  111019.98  111693.47  110889.29  111312.27  25.81775
1756893600000  111312.27  116404.40  111044.05  111318.83  20.18611

[800 rows x 5 columns]


In [7]:
import numpy as np 
import pandas as pd 
import talib

def fet_Ext(df):
    df['price_change'] =  df['close'].pct_change()#15 min change 
    df['price_change_5'] =  df['close'].pct_change(5)#change in 75 mins
    price_range=(df['high'].rolling(14).max()-df['low'].rolling(14).min())
    df['pos_range_prc'] = df['close']-df['low'].rolling(14).min()/price_range
    
    df['rsi'] = talib.RSI(df['close'].values,14)
    df['rsi_oversold'] = (df['rsi'] < 30).astype(int)# stock sold too much will rise
    df['rsi_overbought'] = (df['rsi'] > 80).astype(int)# stock bought too much will fall

    #moving AVGS
    df['smal_10'] = df['close'].rolling(10).mean()# Short term mean
    df['smal_20'] = df['close'].rolling(20).mean()# Long term mean
    df['price_above_smal_10'] = (df['close']>df['smal_10']).astype(int)
    df['price_above_smal_20'] = (df['close']>df['smal_20']).astype(int)
    df['sma_trend'] = (df['smal_10'] > df['smal_20']).astype(int)

    
    df['volume_avg'] = df['volume'].rolling(20).mean()
    df['volume_high'] = (df['volume'] > df['volume_avg'] * 1.5).astype(int)



    df['volatility'] = df['price_change'].rolling(10).std()
    df['high_volatility'] = (df['volatility'] > df['volatility'].rolling(50).mean()).astype(int)

    df['future_return'] = df['price_change'].shift(-1)
    df['target'] = np.where(df['future_return'] > 0.002, 1, 
                            np.where(df['future_return'] < -0.002, -1, 0))
    
    return df.fillna(0)

In [8]:
df = fet_Ext(df)

In [9]:
df

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1754017200000,115966.12,116464.26,115621.74,115652.51,5.20842,0.000000,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,0.000522,0
1754020800000,115648.03,116223.26,88888.00,115712.88,9.60859,0.000522,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.001458,0
1754024400000,115695.53,116255.26,33000.00,115544.20,15.03012,-0.001458,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.003355,-1
1754028000000,115544.20,200000.00,69374.87,115156.52,9.05393,-0.003355,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.000575,0
1754031600000,115156.52,120000.00,77777.00,115090.33,4.78754,-0.000575,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.003774,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756879200000,110685.57,110984.83,110473.86,110719.99,22.30021,0.000311,-0.004110,110709.021262,51.195856,0,...,110870.4575,0,0,1,17.899142,0,0.001690,0,0.003431,1
1756882800000,110723.99,111962.80,110520.18,111099.88,26.37469,0.003431,-0.002443,111089.134020,55.887024,0,...,110939.7860,1,1,1,18.644859,0,0.002018,0,-0.000719,0
1756886400000,111099.88,200000.00,110803.87,111019.99,35.81016,-0.000719,-0.000324,111018.940173,54.696326,0,...,111050.9860,0,0,1,19.258586,1,0.002005,0,0.002633,1


In [10]:
feat_df = fet_Ext(df)

In [11]:
len(df)

800

# Train Model -> Random Forest

In [13]:
from Config import Configur

ImportError: cannot import name 'Configur' from 'Config' (c:\Users\Com\OneDrive\Documents\GitHub\Crypto_bot\Config.py)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pickle


MIN_DATA_POINTS = 300
class RandomForestStrat:
    def __init__(self):
        self.name = "Random_forest_classifier"
        self.model = RandomForestClassifier(
            n_estimators = 50,
            max_depth=8,
            random_state= 42,
            class_weight='balanced'
        )
        self.features = fet_Ext
        self.feature_list = [
        'price_change', 'price_change_5', 'pos_range_prc',
        'rsi', 'rsi_oversold', 'rsi_overbought',
        'price_above_smal_10', 'price_above_smal_20', 'sma_trend',
        'volume_high', 'high_volatility']
        self.is_trained = False

    def Train(self,df):
        df = fet_Ext(df)
        feature_columns =  self.feature_list+ ['target']
        data = df[feature_columns].dropna()
        if len(data) < MIN_DATA_POINTS:
            #More data points are required
            return False 
        X = data[self.feature_list]
        Y = data['target']

        self.model.fit(X,Y)

        y_pred = self.model.predict(X)
        accuracy = accuracy_score(Y,y_pred)

        print(f"Accuracy:{accuracy}")
        with open("Weighted_model.pkl", "wb") as f:
            pickle.dump(self.model, f)

        self.is_trained = True
        return True
    
    def predict(self, df):
        """Make prediction"""
        if not self.is_trained:
            return 'hold', 0.0
        
        # Create features for latest data
        df = self.features(df)
        latest_data = df[self.feature_list].iloc[-1:].fillna(0)
        
        # Get prediction and confidence
        prediction = self.model.predict(latest_data)[0]
        probabilities = self.model.predict_proba(latest_data)[0]
        confidence = max(probabilities)
        
        # Convert to signal
        if confidence > 0.8:
            if prediction == 1:
                return 'buy', confidence
            elif prediction == -1:
                return 'sell', confidence
        
        return 'hold', confidence

In [14]:
mod = RandomForestStrat()

In [15]:
model  = mod.Train(df)

Accuracy:0.885


# Test It out

In [12]:
df[:1]

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1754017200000,115966.12,116464.26,115621.74,115652.51,5.20842,NaN,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,0.000522,0


In [13]:
df.tail()

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1756879200000,110685.57,110984.83,110473.86,110719.99,22.30021,0.000311,-0.004110,110709.021262,51.195856,0,...,110870.4575,0,0,1,17.899142,0,0.001690,0,0.003431,1
1756882800000,110723.99,111962.80,110520.18,111099.88,26.37469,0.003431,-0.002443,111089.134020,55.887024,0,...,110939.7860,1,1,1,18.644859,0,0.002018,0,-0.000719,0
1756886400000,111099.88,200000.00,110803.87,111019.99,35.81016,-0.000719,-0.000324,111018.940173,54.696326,0,...,111050.9860,0,0,1,19.258586,1,0.002005,0,0.002633,1
1756890000000,111019.98,111693.47,110889.29,111312.27,25.81775,0.002633,0.002542,111311.220173,58.204726,0,...,111059.9090,1,1,1,18.379497,0,0.002194,0,0.000059,0
1756893600000,111312.27,116404.40,111044.05,111318.83,20.18611,0.000059,0.005721,111317.780173,58.282814,0,...,111057.4510,1,1,1,18.317575,0,0.002165,0,NaN,0


In [14]:
df[-1:]

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1756893600000,111312.27,116404.4,111044.05,111318.83,20.18611,0.000059,0.005721,111317.780173,58.282814,0,...,111057.451,1,1,1,18.317575,0,0.002165,0,NaN,0


In [15]:
import pickle

with open("Weighted_model.pkl", "rb") as f:
    models = pickle.load(f)
test_df = df[-100:]
test_df = test_df.drop(['target'], axis=1)
feature_list = [
'price_change', 'price_change_5', 'pos_range_prc',
'rsi', 'rsi_oversold', 'rsi_overbought',
'price_above_smal_10', 'price_above_smal_20', 'sma_trend',
'volume_high', 'high_volatility']
X = test_df[feature_list]
dictions  = models.predict(X)

In [16]:
from sklearn.metrics import accuracy_score
res_df = pd.DataFrame()
res_df['results'] = dictions
accuracy = accuracy_score(res_df['results'],df['target'][-100:])
print(accuracy)

0.83


In [17]:
df['target'].value_counts()

target
 0    456
 1    178
-1    166
Name: count, dtype: int64

Model result comes in -1 -sell , 0 -hold ,1 buy

In [18]:
dictions[-20:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0])

In [19]:
side = {-1:"sell",0:"hold",1:"buy"}

# Prepping DF for model 

In [20]:
df.index

Index([1754017200000, 1754020800000, 1754024400000, 1754028000000,
       1754031600000, 1754035200000, 1754038800000, 1754042400000,
       1754046000000, 1754049600000,
       ...
       1756861200000, 1756864800000, 1756868400000, 1756872000000,
       1756875600000, 1756879200000, 1756882800000, 1756886400000,
       1756890000000, 1756893600000],
      dtype='int64', name='timestamp', length=800)

In [21]:
df.index = pd.to_datetime(df.index, unit="ms")

In [22]:
df

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-08-01 03:00:00,115966.12,116464.26,115621.74,115652.51,5.20842,NaN,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,0.000522,0
2025-08-01 04:00:00,115648.03,116223.26,88888.00,115712.88,9.60859,0.000522,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,-0.001458,0
2025-08-01 05:00:00,115695.53,116255.26,33000.00,115544.20,15.03012,-0.001458,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,-0.003355,-1
2025-08-01 06:00:00,115544.20,200000.00,69374.87,115156.52,9.05393,-0.003355,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,-0.000575,0
2025-08-01 07:00:00,115156.52,120000.00,77777.00,115090.33,4.78754,-0.000575,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,-0.003774,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-03 06:00:00,110685.57,110984.83,110473.86,110719.99,22.30021,0.000311,-0.004110,110709.021262,51.195856,0,...,110870.4575,0,0,1,17.899142,0,0.001690,0,0.003431,1
2025-09-03 07:00:00,110723.99,111962.80,110520.18,111099.88,26.37469,0.003431,-0.002443,111089.134020,55.887024,0,...,110939.7860,1,1,1,18.644859,0,0.002018,0,-0.000719,0
2025-09-03 08:00:00,111099.88,200000.00,110803.87,111019.99,35.81016,-0.000719,-0.000324,111018.940173,54.696326,0,...,111050.9860,0,0,1,19.258586,1,0.002005,0,0.002633,1


In [23]:
%load_ext autoreload
%autoreload 2

In [14]:
import sys
import os

# go up two levels (from Method/RF File/) and into Pipeline
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../Pipeline")))

from GetData import Manage_data

# Using Model in latest Data

In [16]:


import asyncio
import pickle
from datetime import datetime

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../Pipeline")))


from GetData import Manage_data
from Manage_orders import ExecutionEngine
from Logger import TradeLogger
from Visualizer import Visualizerr
from config import Configs
from FeatureExtract import Features
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import dash
from dash import dcc, html, Input, Output
import threading


class CryptoTradingBot:
    def __init__(self, df=None):
        self.data_manager = Manage_data()
        self.execution_engine = ExecutionEngine()
        self.logger = TradeLogger()
        self.visualizer = Visualizerr()
        self.last_timestamp = None
        self.running = False
        self.Extractor = Features()
        self.feature_list  =[
        'price_change', 'price_change_5', 'pos_range_prc',
        'rsi', 'rsi_oversold', 'rsi_overbought',
        'price_above_smal_10', 'price_above_smal_20', 'sma_trend',
        'volume_high', 'high_volatility']
        self.position = None
        self.test_df = df  # premade DataFrame
        with open("Weighted_model.pkl", "rb") as f:
            self.models = pickle.load(f)

    def run_bot(self):
        """Main bot execution loop"""
        self.running = True
        
        print(f"Starting crypto trading bot for {Configs.SYMBOL}")
        print(f"Strategy: Random Forest ")
        print(f"Initial Balance: ${Configs.INIT_BALANCE}")
        print("-" * 50)
        

        for i in range( 100, len(self.test_df)-1):  # start after enough candles for indicators
            try:
                df = self.test_df.iloc[: i+1].copy()

                clean_df = df.dropna()
                Ext_Features = self.Extractor.fet_Ext(clean_df)
                feat_list = self.Extractor.feature_list
                clean_df = Ext_Features[feat_list]
                
                self.signal = self.models.predict(clean_df)
                
                self.side = {-1: "sell", 0: "hold", 1: "buy"}

                current_price = df["close"].iloc[-1]
                current_time = df.index[-1]
                # print(self.side[self.signal[-1]])

                # # Check exit conditions
                self.execution_engine.check_exit_conditions(current_price, current_time)

                # # Execute new signal
                self.execution_engine.execute_signal(self.side[self.signal[-1]], current_price, current_time)

                # Log completed trades
                if self.execution_engine.trades:
                    for trade in self.execution_engine.trades:
                        if trade not in getattr(self, 'logged_trades', []):
                            self.logger.log_trade(trade)
                            if not hasattr(self, 'logged_trades'):
                                self.logged_trades = []
                            self.logged_trades.append(trade)

                # Display status
                self.display_status(df, self.signal, current_price)

            except Exception as e:
                print(f"Error in backtest loop: {e}")
                break
    def display_status(self, df, signal, price):
        """Display current bot status"""
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        rsi = df['rsi'].iloc[-1] if 'rsi' in df.columns else 0
        
        print(f"\n[{current_time}] Price: ${price:.2f} | RSI: {rsi:.1f} | Signal: {signal}")
        
        if self.execution_engine.position:
            pos = self.execution_engine.position
            unrealized_pnl = pos.calculate_pnl(price)
            print(f"Position: {pos.side.upper()} | Size: {pos.size:.4f} | Entry: ${pos.entry_price:.2f} | Unrealized P&L: ${unrealized_pnl:.2f}")
        
        print(f"Balance: ${self.execution_engine.balance:.2f} | Total Trades: {len(self.execution_engine.trades)}")
    def start_dashboard(self):
        """Start web dashboard in separate thread"""
        bot_ref =  self
        def run_dash():
            # Setup callbacks for live updates
            
            @self.visualizer.app.callback(
                [Output('price-chart', 'figure'),
                 Output('equity-chart', 'figure'),
                 Output('metrics-display', 'children')],
                [Input('interval-component', 'n_intervals')]
            )
            def update_dashboard(n_intervals):
                # Get latest data

                print(f"Dashboard updating... iteration {n_intervals}")  # Debug
                
                df = bot_ref.current_df.copy() if hasattr(bot_ref, 'current_df') and bot_ref.current_df is not None else bot_ref.test_df.copy()
                df = bot_ref.Extractor.fet_Ext(df)
                
                price_fig = self.visualizer.create_price_chart(
                    df, 
                    self.execution_engine.trades, 
                    df['close'].iloc[-1] if len(df) > 0 else 0
                )
                
                equity_fig = self.visualizer.create_equity_curve(
                    self.execution_engine.equity_curve
                )
                
                # Create metrics display
                if len(self.execution_engine.trades) > 0:
                    metrics = self.logger.calculate_metrics(self.execution_engine.trades)
                    metrics_div = html.Div([
                        html.H3("Performance Metrics"),
                        html.P(f"Win Rate: {metrics['win_rate']:.1f}%"),
                        html.P(f"Total P&L: ${metrics['total_pnl']:.2f}"),
                        html.P(f"Total Trades: {metrics['total_trades']}"),
                        html.P(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}"),
                        html.P(f"Max Drawdown: ${metrics['max_drawdown']:.2f}")
                    ])
                else:
                    metrics_div = html.Div([html.H3("Waiting for trades...")])
                
                return price_fig, equity_fig, metrics_div
            
            self.visualizer.app.run(debug=False, host='127.0.0.1', port=8050)
        
        dashboard_thread = threading.Thread(target=run_dash)
        dashboard_thread.daemon = True
        dashboard_thread.start()
        print("Dashboard started at http://127.0.0.1:8050")

ModuleNotFoundError: No module named 'Pipeline'

In [ ]:
# Load or create your test DataFrame
# df = pd.read_csv("your_test_data.csv", index_col=0, parse_dates=True)

# Pass it into the bot
bot = CryptoTradingBot(df=df)
# Start dashboard
bot.start_dashboard()

# Wait a moment for dashboard to start
time.sleep(2)

# Run trading bot
bot.run_bot()

Dashboard started at http://127.0.0.1:8050


Dashboard updating... iteration 0
Visualizer got df columns: Index(['open', 'high', 'low', 'close', 'volume', 'price_change',
       'price_change_5', 'pos_range_prc', 'rsi', 'rsi_oversold',
       'rsi_overbought', 'smal_10', 'smal_20', 'price_above_smal_10',
       'price_above_smal_20', 'sma_trend', 'volume_avg', 'volume_high',
       'volatility', 'high_volatility', 'future_return', 'target'],
      dtype='object')
Shape: (800, 22)
Starting crypto trading bot for BTC/USDT
Strategy: Random Forest 
Initial Balance: $10000
--------------------------------------------------
OPENED BUY position: 0.0439 @ $113969.71

[2025-09-03 16:23:31] Price: $113969.71 | RSI: 38.1 | Signal: [-1 -1  1 -1 -1 -1 -1 -1  1  0  1 -1  1 -1  0  0  0  0  1  0 -1  1  0  0
  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
 -1  0  0  0  0  0  0  0  0  1]
Position: BUY | Size: 0.0439 | Entry: $113969.71 | Unrealized P&

Dashboard updating... iteration 34
Visualizer got df columns: Index(['open', 'high', 'low', 'close', 'volume', 'price_change',
       'price_change_5', 'pos_range_prc', 'rsi', 'rsi_oversold',
       'rsi_overbought', 'smal_10', 'smal_20', 'price_above_smal_10',
       'price_above_smal_20', 'sma_trend', 'volume_avg', 'volume_high',
       'volatility', 'high_volatility', 'future_return', 'target'],
      dtype='object')
Shape: (800, 22)
Dashboard updating... iteration 35
Visualizer got df columns: Index(['open', 'high', 'low', 'close', 'volume', 'price_change',
       'price_change_5', 'pos_range_prc', 'rsi', 'rsi_oversold',
       'rsi_overbought', 'smal_10', 'smal_20', 'price_above_smal_10',
       'price_above_smal_20', 'sma_trend', 'volume_avg', 'volume_high',
       'volatility', 'high_volatility', 'future_return', 'target'],
      dtype='object')
Shape: (800, 22)
Dashboard updating... iteration 36
Visualizer got df columns: Index(['open', 'high', 'low', 'close', 'volume', 'price_